In [1]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    LabelToMaskd,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.apps import download_and_extract
from monai.data import ITKReader
import torch
import matplotlib.pyplot as plt
import tempfile
import os
import glob
from torchvision.utils import save_image
import numpy as np
from datetime import datetime
from skimage import io
from PIL import Image, ImageSequence

In [2]:
def get_data(root_dir, resource, name: str):

    compressed_file = os.path.join(root_dir, name)
    
    if not os.path.exists(compressed_file):
        download_and_extract(resource, compressed_file, root_dir)

In [20]:
def breakdownstack(files, path):
    for file in files:
        filename_ext = os.path.basename(file)
        filename = os.path.splitext(filename_ext)[0]
        try:
            im = Image.open(file)
            for i, page in enumerate(ImageSequence.Iterator(im)):
                if not os.path.isfile(f'{path}/{filename}_{i}.png'):
                    try:
                        page.save(f'{path}/{filename}_{i}.png')
                    except:
                        print(f"error - {filename}_{i}.png")
        except:
            print(f"error - {filename}")


In [21]:
def check3d(path):
    files = glob.glob(path + '/*.tif')

    if len(files) > 1:
        breakdownstack(files, path)
    else:
        print("No 3D data found")

In [23]:
directory = os.environ.get('MONAI_DATA_DIRECTORY', "MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
data_dir = os.path.join(root_dir, "data")

#get_data(root_dir, "https://documents.epfl.ch/groups/c/cv/cvlab-unit/www/data/%20ElectronMicroscopy_Hippocampus/volumedata.tif", "lucchi.zip")

In [22]:
for f in os.scandir(data_dir):
    if f.path != 'MONAI_DATA_DIRECTORY/.DS_Store':
        check3d(f.path)

MONAI_DATA_DIRECTORY/lucchi
